In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import gc
import warnings
warnings.filterwarnings("ignore")


In [ ]:

# STEP 1: LOAD DATA
application_train = pd.read_csv('/content/drive/My Drive/group6_Final_DP/data/raw_data/application_train.csv')
POS_CASH_balance = pd.read_csv('/content/drive/My Drive/group6_Final_DP/data/raw_data/POS_CASH_balance.csv')

# STEP 2: LOẠI BỎ DỮ LIỆU TRÙNG LẶP
print("POS_CASH_balance duplicated rows:", POS_CASH_balance.duplicated().sum())
POS_CASH_balance = POS_CASH_balance.drop_duplicates()

# STEP 3: XỬ LÝ GIÁ TRỊ THIẾU
missing_pcb = POS_CASH_balance.isnull().sum() / len(POS_CASH_balance) * 100
print("Missing values in POS_CASH_balance:\n", missing_pcb[missing_pcb > 0])

# Điền giá trị thiếu hợp lý
POS_CASH_balance['SK_DPD'] = POS_CASH_balance['SK_DPD'].fillna(0)
POS_CASH_balance['SK_DPD_DEF'] = POS_CASH_balance['SK_DPD_DEF'].fillna(0)

# STEP 4: TẠO CÁC TÍNH NĂNG TỪ POS_CASH_BALANCE
# Tổng số khoản vay
pcb_prev_count = POS_CASH_balance.groupby('SK_ID_CURR')['SK_ID_PREV'].nunique()

# Số tháng trung bình
pcb_avg_month = POS_CASH_balance.groupby('SK_ID_CURR')['MONTHS_BALANCE'].count() / pcb_prev_count

# Thời gian hoạt động gần nhất
pcb_recent_active = POS_CASH_balance.groupby('SK_ID_CURR')['MONTHS_BALANCE'].max()

# Số lần thay đổi số tiền trả góp
pcb_temp_inst_change_time = POS_CASH_balance[['SK_ID_PREV', 'CNT_INSTALMENT']].groupby('SK_ID_PREV')['CNT_INSTALMENT'].nunique() \
                            .map(lambda x: x - 1).reset_index().rename(columns={'CNT_INSTALMENT': 'pcb_prev_inst_change_time'})

pcb_temp = POS_CASH_balance.groupby(['SK_ID_CURR', 'SK_ID_PREV'])['MONTHS_BALANCE'].count().reset_index()
pcb_temp = pcb_temp.merge(pcb_temp_inst_change_time, on='SK_ID_PREV')
pcb_inst_change_time = pcb_temp.groupby('SK_ID_CURR')['pcb_prev_inst_change_time'].sum()

# Số tiền trả góp trung bình
pcb_temp = POS_CASH_balance.groupby(['SK_ID_CURR', 'SK_ID_PREV'])['CNT_INSTALMENT'].mean().reset_index()
pcb_avg_inst = pcb_temp.groupby(['SK_ID_CURR'])['CNT_INSTALMENT'].mean()

# Khoản vay còn hoạt động
pcb_temp = POS_CASH_balance.groupby(['SK_ID_CURR', 'SK_ID_PREV'])['MONTHS_BALANCE'].max().reset_index()
pcb_temp = pcb_temp.merge(POS_CASH_balance[['SK_ID_CURR', 'SK_ID_PREV', 'MONTHS_BALANCE', 'NAME_CONTRACT_STATUS']],
                          on=['SK_ID_CURR', 'SK_ID_PREV', 'MONTHS_BALANCE'])
pcb_temp['active_1'] = pcb_temp.MONTHS_BALANCE.map(lambda x: 1 if x >= -4 else 0)
pcb_temp['active_2'] = pcb_temp.NAME_CONTRACT_STATUS.map(lambda x: 1 if x == 'Active' else 0)
pcb_temp['active'] = pcb_temp.active_1 * pcb_temp.active_2
pcb_active_inst = pcb_temp.groupby('SK_ID_CURR')['active'].sum()

# DPD (Days Past Due)
pcb_temp = POS_CASH_balance[['SK_ID_CURR', 'SK_ID_PREV', 'SK_DPD', 'SK_DPD_DEF']].groupby(['SK_ID_CURR', 'SK_ID_PREV'])[['SK_DPD', 'SK_DPD_DEF']].max().reset_index()
pcb_max_dpd_days = pcb_temp.groupby('SK_ID_CURR')['SK_DPD'].max()
pcb_total_dpd_days = pcb_temp.groupby('SK_ID_CURR')['SK_DPD'].sum()
pcb_max_largedpd_days = pcb_temp.groupby('SK_ID_CURR')['SK_DPD_DEF'].max()
pcb_total_largedpd_days = pcb_temp.groupby('SK_ID_CURR')['SK_DPD_DEF'].sum()

# DPD trong 1 năm
pcb_temp = POS_CASH_balance.loc[POS_CASH_balance.MONTHS_BALANCE >= -12].groupby(['SK_ID_CURR', 'SK_ID_PREV'])[['SK_DPD', 'SK_DPD_DEF']].max().reset_index()
pcb_total_dpd_days_1y = pcb_temp.groupby('SK_ID_CURR')['SK_DPD'].sum()

# STEP 5: KẾT HỢP DỮ LIỆU
pcb_num = pd.DataFrame({
    'pcb_prev_count': pcb_prev_count,
    'pcb_avg_month': pcb_avg_month,
    'pcb_recent_active': pcb_recent_active,
    'pcb_inst_change_time': pcb_inst_change_time,
    'pcb_avg_inst': pcb_avg_inst,
    'pcb_active_inst': pcb_active_inst,
    'pcb_max_dpd_days': pcb_max_dpd_days,
    'pcb_total_dpd_days': pcb_total_dpd_days,
    'pcb_max_largedpd_days': pcb_max_largedpd_days,
    'pcb_total_largedpd_days': pcb_total_largedpd_days,
    'pcb_total_dpd_days_1y': pcb_total_dpd_days_1y
}).reset_index()

# Khoản vay không có DPD hoặc DPD lớn
pcb_feature = pcb_num.copy()
pcb_feature['pcb_no_dpd'] = pcb_feature['pcb_total_dpd_days'].map(lambda x: 0 if x > 0 else 1)
pcb_feature['pcb_no_largedpd'] = pcb_feature['pcb_total_largedpd_days'].map(lambda x: 0 if x > 0 else 1)

# STEP 6: LƯU DỮ LIỆU
pcb_feature.to_csv('/content/drive/My Drive/group6_Final_DP/data/processed_data/pcb_feature.csv', index=False)
print("POS_CASH_balance features saved.")


POS_CASH_balance duplicated rows: 0
Missing values in POS_CASH_balance:
 CNT_INSTALMENT           0.255906
CNT_INSTALMENT_FUTURE    0.256081
dtype: float64
POS_CASH_balance features saved.
